In [ ]:
from numpy import *
import numpy.matlib as matlib
import itertools
import sys

In [ ]:
DEBUG = True

GRID_HEIGHT = 4
GRID_WIDTH = 4

<img src="/files/4x4%20Grid%20-%20State%20Transistion%20Diagram.png"/>

In [ ]:
class Action:
    value_map = {'up':0, 'down':1, 'left':2, 'right':3}
    
    def __init__(self, value):
        self.value = value.lower()
        self.index = self.value_map[self.value]
       
    def apply_to(self, state):
        new_i = state.i
        new_j = state.j
            
        if self.value == 'up':
            new_i = state.i - 1 if state.i > 0 else state.i
        elif self.value == 'down':
            new_i = state.i + 1 if state.i + 1 < GRID_HEIGHT else state.i
        elif self.value == 'left':
            new_j = state.j - 1 if state.j > 0 else state.j
        elif self.value == 'right':
            new_j = state.j + 1 if state.j + 1 < GRID_WIDTH else state.j
        
        return State(new_i, new_j)
            
    def __eq__(self, other):
        if self.index == other.index:
            return True
        return False
    
    def __str__(self):
        return self.value

In [ ]:
class State:
    def __init__(self, i, j):
        self.i = i
        self.j = j
        self.index = self.i * GRID_HEIGHT + self.j
   
    def left_of(self, other):
        if self.i == other.i and self.j - other.j == -1:
            return True
        return False

    def right_of(self, other):
        if self.i == other.i and self.j - other.j == 1:
            return True
        return False
        
    def above(self, other):
        if self.j == other.j and self.i - other.i == -1:
            return True
        return False

    def below(self, other):
        if self.j == other.j and self.i - other.i == 1:
            return True
        return False
    
    def on_top_edge(self):
        if self.i == 0:
            return True
        
    def on_bottom_edge(self):
        if self.i == GRID_HEIGHT - 1:
            return True
        
    def on_left_edge(self):
        if self.j == 0:
            return True
        
    def on_right_edge(self):
        if self.j == GRID_WIDTH - 1:
            return True
        
    def __eq__(self, other):
        if self.index == other.index:
            return True
        return False
    
    def __str__(self):
        return 's{}{}'.format(self.i,self.j)

In [ ]:
actions = [Action('up'), Action('down'), Action('left'), Action('right')]
states = [State(i,j) for i,j in itertools.product(range(GRID_HEIGHT), range(GRID_WIDTH))]

# terminal states
terminal_states = [State(0,0), State(3,3)]

In [ ]:
# |S| x |A|
uni_random_policy = full(shape=(len(states), len(actions)), fill_value=0.25)

In [ ]:
def trans(s, a, s_p):
    if s in terminal_states:
        return 0.0
    if a == Action('up'):
        if s_p.above(s) or (s == s_p and s.on_top_edge()):
            return 1.0
    elif a == Action('down'):
        if s_p.below(s) or (s == s_p and s.on_bottom_edge()):
            return 1.0
    elif a == Action('left'):
        if s_p.left_of(s) or (s == s_p and s.on_left_edge()):
            return 1.0
    elif a == Action('right'):
        if s_p.right_of(s) or (s == s_p and s.on_right_edge()):
            return 1.0

    return 0.0
    
# |S| x |A| x |S|
p_trans = zeros(shape=(len(states), len(actions), len(states)))

for s, a, s_p in itertools.product(states, actions, states):
    p_trans[s.index, a.index, s_p.index] = trans(s, a, s_p) 

In [ ]:
r_term = 0.0  # Reward for terminal state
r_step = -1.0 # Reward for any non-terminal state

gamma = 0.95  # Discount factor

In [ ]:
def reward(state, action, next_state):
    if state in terminal_states:
        return r_term
    else:
        return r_step

# |S| x |A| x |S|
r = zeros(shape=(len(states),len(actions),len(states)))

for s, state in enumerate(states):
    for a, action in enumerate(actions):
        for s_p, next_state in enumerate(states):
            r[s,a,s_p] = reward(state,action,next_state)

In [ ]:
# Add a new parameter for vk_new (this will allow modified version and non-modified version based on call)
def policy_evaluation(policy, vk):
    vk_new = zeros(shape=(len(states)))
    for s, state in enumerate(states):
        for a, action in enumerate(actions):
            for s_p, next_state in enumerate(states):
                vk_new[s] += policy[s, a] * p_trans[s, a, s_p] * (r[s, a, s_p] + gamma * vk[s_p])
    return vk_new

Evaluating Uniform Random Policy

In [ ]:
vk = zeros(shape=(len(states)))

NUM_ITERS = 500
for k in range(NUM_ITERS):
    vk = policy_evaluation(uni_random_policy, vk)

vk_uni = copy(vk)
for s in states:
    print '{} = {}'.format(s, vk_uni[s.index])

In [ ]:
from random import choice

def random_action():
    return choice(actions)

def random_state():
    return choice(states)

def discount(time):
    if time == 0:
        return 1
    
    return gamma ** time

In [ ]:
class Event(object):
    def __init__(self, state, action=None, next_state=None):
        self.state = state
        self.action = action
        self.next_state = next_state

    def __str__(self):
        return '{},{},{}'.format(self.state, self.action, self.next_state)
        
def generate_episode():
    
    # Add random initial state
    events = [Event(state=random_state())]

    while events[-1].state not in terminal_states:
        events[-1].action = random_action()
        events[-1].next_state = events[-1].action.apply_to(events[-1].state)
        
        events.append(Event(state=events[-1].next_state))

    return events

In [ ]:
def calculate_return(trajectory):
    value = 0.0
    
    for event, t in zip(trajectory, range(len(trajectory))):
        value += gamma**t * reward(event.state, event.action, event.next_state)
        
    return value

def no_progress(V1, V2, theta):
    if not (V1 and V2):
        return False
    
    diffs = [abs(v1 - v2) for v1, v2 in zip(V1, V2)]
    if max(diffs) < theta:
        return True
    
    return False

# Monte-Carlo Prediction

## Two methods:

1. First Visit Monte-Carlo Prediction
2. Every Visit Monte-Carlo Prediction

In [ ]:
from collections import namedtuple

Result = namedtuple('Result', ['state_values', 'iters'])

# Monte Carlo Algorithms
FIRST_VISIT = 0
EVERY_VISIT = 1

def monte_carlo_prediction(algorithm, max_iters=1e6, max_no_progress=100, theta=1.0e-3):

    # state visit counters (one element per state)
    N = [0] * len(states)

    # total returns (one element per state)
    S = [0.0] * len(states)
    
    # state-value function approximation
    V = [0.0] * len(states)
    V_old = None
    
    iters = 0
    iters_no_progress = 0
    
    while not (iters >= max_iters or iters_no_progress >= max_no_progress):
        
        if no_progress(V, V_old, theta):
            iters_no_progress += 1
        else:
            iters_no_progress = 0
            
        # states already visited in this episode
        visited = []
        V_old = list(V)
        
        episode = generate_episode()
        for i, event in enumerate(episode):
        
            # "First-Visit Monte Carlo" only includes returns 
            # when a state is first visited.  Other visits are bypassed.
            if algorithm == FIRST_VISIT and event.state in visited:
                continue

            visited.append(event.state)

            # Trajectory includes all states visited from this point
            # until the end of the episode
            trajectory = episode[i:]
            
            N[event.state.index] += 1
            S[event.state.index] += calculate_return(trajectory)
            V[event.state.index] = S[event.state.index] / N[event.state.index]
                               
        iters += 1

    return Result(state_values=V, iters=iters)

In [ ]:
result = monte_carlo_prediction(algorithm=FIRST_VISIT, max_iters=100000, theta=1.0e-4)
     
for v in result.state_values:
    print v
    
print "iters: ", result.iters

In [ ]:
result = monte_carlo_prediction(algorithm=EVERY_VISIT, max_iters=100000, theta=1.0e-4)
     
for v in result.state_values:
    print v
    
print "iters: ", result.iters

In [ ]:
def incremental_monte_carlo_prediction(algorithm, max_iters=1e6, max_no_progress=100, theta=1.0e-3, alpha=0.001):
   
    # state-value function approximation
    V = [0.0] * len(states)
    V_old = None
    
    iters = 0
    iters_no_progress = 0
    
    while not (iters >= max_iters or iters_no_progress >= max_no_progress):
        
        if no_progress(V, V_old, theta):
            iters_no_progress += 1
        else:
            iters_no_progress = 0
            
        # states already visited in this episode
        visited = []
        V_old = list(V)
        
        episode = generate_episode()
        for i, event in enumerate(episode):
        
            s = event.state
            
            # "First-Visit Monte Carlo" only includes returns 
            # when a state is first visited.  Other visits are bypassed.
            if algorithm == FIRST_VISIT:
                if s in visited:
                    continue

            visited.append(s)

            # Trajectory includes all states visited from this point
            # until the end of the episode
            trajectory = episode[i:]

            Gt = calculate_return(trajectory)
            
            V[s.index] = V[s.index] + alpha * (Gt - V[s.index])
                               
        iters += 1

    return Result(state_values=V, iters=iters)

In [ ]:
result = incremental_monte_carlo_prediction(algorithm=FIRST_VISIT, max_iters=1000000, theta=1.0e-4)
     
for v in result.state_values:
    print v
    
print "iters: ", result.iters

In [ ]:
result = incremental_monte_carlo_prediction(algorithm=EVERY_VISIT, max_iters=100000, theta=1.0e-4)
     
for v in result.state_values:
    print v
    
print "iters: ", result.iters

In [ ]:
import numpy as np

def root_mean_sqr_err(expected, actual):
    return np.sqrt(((expected - actual)**2).mean)